In [ ]:
import pandas as pd
import catboost as cb
from datetime import date
from dateutil.relativedelta import relativedelta

In [ ]:
# лоадим трейн + доп трейн

train = pd.read_csv('/kaggle/input/hackaton-dc/train_dataset_Дипстрой/dataset_ksg.csv')
attr = pd.read_csv('/kaggle/input/hackaton-dc/train_dataset_Дипстрой/attr.csv')

# создаем дикт с задачами, сдвиги по которым будем предсказывать

tasks_ex = pd.read_excel('/kaggle/input/hackaton-dc/train_dataset_Дипстрой/.xlsx')
tasks = pd.Series(tasks_ex['НазваниеЗадачи'].values,index=tasks_ex['Кодзадачи']).to_dict()

In [ ]:
# нужно выкинуть все NaN значения по понятным причинам (если тут отсутсвтует либо дата начала, либо
# дата окончания, то предсказание невозможно)

train = train.dropna(subset=['ДатаНачалаЗадачи', 'ДатаОкончанияЗадачи','ДатаначалаБП0', 'ДатаокончанияБП0'])

In [ ]:
# переводим строчки в формат даты ДатаначалаБП0

train['ДатаНачалаЗадачи'] = pd.to_datetime(train['ДатаНачалаЗадачи'])
train['ДатаОкончанияЗадачи'] = pd.to_datetime(train['ДатаОкончанияЗадачи'])
train['ДатаначалаБП0'] = pd.to_datetime(train['ДатаначалаБП0'])
train['ДатаокончанияБП0'] = pd.to_datetime(train['ДатаокончанияБП0'])

# переводим в юникс время

train['ДатаНачалаЗадачи'] = train['ДатаНачалаЗадачи'].apply(lambda x: x.timestamp())
train['ДатаОкончанияЗадачи'] = train['ДатаОкончанияЗадачи'].apply(lambda x: x.timestamp())
train['ДатаначалаБП0'] = train['ДатаначалаБП0'].apply(lambda x: x.timestamp())
train['ДатаокончанияБП0'] = train['ДатаокончанияБП0'].apply(lambda x: x.timestamp())

# создаем колонку таргета. Идея такая: предсказываем таргет (= время в секундах от начала до конца задачи)
# переводим секунды в дни, прибавляем к дате начала задачи семпла - получаем ответ


# target перевожу в дни - будем предиктить число дней
# фича plan_time - кол-во дней между плановой датой окончания и начала по договору

train['fact_time'] = 0
train['fact_time'] = (train['ДатаОкончанияЗадачи'] - train['ДатаНачалаЗадачи'])/60/60/24
train['plan_time'] = 0
train['plan_time'] = (train['ДатаокончанияБП0'] - train['ДатаначалаБП0'])/60/60/24

train['target']=(train['ДатаОкончанияЗадачи'] - train['ДатаокончанияБП0'])/60/60/24

Всего 28 уникальных объектов и в аттрибутах есть лишние объекты:

In [ ]:
print(len(train['obj_key'].unique()))
print(len(attr['obj_key'].unique()))

list_of_objects = train['obj_key'].unique()

Вытянем из аттрибутов доп фичи:

In [ ]:
train['состояние площадки'] = ' '
train['Площадь'] = 0
train['Генпроектировщик'] = 0
train['Генподрядчик'] = 0
train['Кол-во рабочих'] = 0

for obj in list_of_objects:
    train.loc[train['obj_key']==obj,'состояние площадки'] = attr[attr['obj_key']==obj]['состояние площадки'].values[0]
    train.loc[train['obj_key']==obj,'Площадь'] = attr[attr['obj_key']==obj]['Площадь'].values[0]
    train.loc[train['obj_key']==obj,'Генпроектировщик'] = attr[attr['obj_key']==obj]['Генпроектировщик'].values[0]
    train.loc[train['obj_key']==obj,'Генподрядчик'] = attr[attr['obj_key']==obj]['Генподрядчик'].values[0]
    train.loc[train['obj_key']==obj,'Кол-во рабочих'] = attr[attr['obj_key']==obj]['Кол-во рабочих'].values[0]

что не будет использоваться:

Фичи 'ДатаНачалаЗадачи', 'ДатаОкончанияЗадачи','ДатаначалаБП0', 'ДатаокончанияБП0', 'Статуспоэкспертизе', 'Экспертиза', 'date_report' - не используем в модели (первые 4 берем, чтобы получить 2 новых фичи, которые уже заюзаем

In [ ]:
train = train[['obj_prg', 'obj_subprg', 'obj_key', 'Кодзадачи',
       'НазваниеЗадачи', 'ПроцентЗавершенияЗадачи','fact_time',
       'plan_time', 'target', 'состояние площадки', 'Площадь',
       'Генпроектировщик', 'Генподрядчик', 'Кол-во рабочих']]

Много дубликатов (повторяются еженедельно отчеты о том, что типо ничего не сдвинулось == все данные одинаковые)

In [ ]:
train.drop_duplicates(inplace = True)

Для каждой из тасок обучим свою модель + поменяем всю фигню, чтобы работало

In [ ]:
tasks_list = list(tasks.values())
tasks_list = [x.lower() for x in tasks_list]
tasks_list[5] = 'утверждение перечня тхо в донм'
train['состояние площадки'] = train['состояние площадки'].fillna('')

In [ ]:
cat_features = ['obj_prg', 'obj_subprg', 'состояние площадки']

# features - итоговый список фичей!

features = ['obj_prg', 'obj_subprg','состояние площадки', 'ПроцентЗавершенияЗадачи','fact_time',
       'plan_time', 'Площадь',
       'Генпроектировщик', 'Генподрядчик', 'Кол-во рабочих']

for cat in cat_features:
    train[cat] = train[cat].astype('string')

train['НазваниеЗадачи'] = train['НазваниеЗадачи'].str.lower()    
    
for task in tasks_list:
    model = cb.CatBoostRegressor()
    print(task)
    
    #Создаем трейн и модель только для каждой из таргетных задач
    
    cb_train = train[train['НазваниеЗадачи'] == task]
    
    model.fit(cb_train[features], cb_train['target'], cat_features = cat_features,  verbose = 0)
    model.save_model('CAT_{}.cbm'.format(task))